In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch
import torch.nn as nn
import pickle
import sys
import imp
sys.path.append('../')
%matplotlib inline

/home/cgagne/cvar_generation/conda_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import (
    GPT2Config,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    MinLengthLogitsProcessor,
    LogitsProcessorList,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from cvar_helpers import calc_cvar_from_quantiles
from cvar_sampler import distort_probabilities
from rl_learner import TD_Learner
from generate_sentences_w_cvar import score_sentiment

import manual_examples
imp.reload(manual_examples)
from manual_examples import get_probabilities, get_distributions, plot_distributions, get_prompt_distribution

### Loading Sentiment Model

In [4]:
device = 'cpu'
sentiment_tokenizer = AutoTokenizer.from_pretrained('../models/pretrained/cardiffnlp-twitter-roberta-base-sentiment')
sentiment_model = AutoModelForSequenceClassification.from_pretrained('../models/pretrained/cardiffnlp-twitter-roberta-base-sentiment')
sentiment_model.to(device);

### Examples

In [5]:
prompts = ['I felt really sick.', 
               'But then I went to the doctors to get medicine.', 
               'Now I feel better.']

compound = ' '.join(prompts)
prompts.append(compound)
for prompt in prompts:
    print(prompt)
sentiments = score_sentiment(prompts, sentiment_tokenizer, sentiment_model, device)
print("")
print(f'individual sentiments: {sentiments[0:-1]}')
print(f'summed: {sentiments[0:-1].sum():0.2f}')
print(f'scored as a whole: {sentiments[-1]:0.2f}')


I felt really sick.
But then I went to the doctors to get medicine.
Now I feel better.
I felt really sick. But then I went to the doctors to get medicine. Now I feel better.

individual sentiments: [-0.9379235  -0.15414045  0.88297855]
summed: -0.21
scored as a whole: 0.61


In [6]:
prompts = ['I felt good.', 
               'But then I went to the restaurant.', 
               'And I got sick.']

compound = ' '.join(prompts)
prompts.append(compound)
for prompt in prompts:
    print(prompt)
sentiments = score_sentiment(prompts, sentiment_tokenizer, sentiment_model, device)
print("")
print(f'individual sentiments: {sentiments[0:-1]}')
print(f'summed: {sentiments[0:-1].sum():0.2f}')
print(f'scored as a whole: {sentiments[-1]:0.2f}')


I felt good.
But then I went to the restaurant.
And I got sick.
I felt good. But then I went to the restaurant. And I got sick.

individual sentiments: [ 0.95712042  0.02555282 -0.91727647]
summed: 0.07
scored as a whole: -0.47


In [7]:
prompts = ['I felt good.', 
               'I felt bad.']

compound = ' '.join(prompts)
prompts.append(compound)
for prompt in prompts:
    print(prompt)
sentiments = score_sentiment(prompts, sentiment_tokenizer, sentiment_model, device)
print("")
print(f'individual sentiments: {sentiments[0:-1]}')
print(f'summed: {sentiments[0:-1].sum():0.2f}')
print(f'scored as a whole: {sentiments[-1]:0.2f}')


I felt good.
I felt bad.
I felt good. I felt bad.

individual sentiments: [ 0.95712024 -0.79355948]
summed: 0.16
scored as a whole: 0.24


In [8]:
prompts = ['I felt good.', 
               'But I felt bad.']

compound = ' '.join(prompts)
prompts.append(compound)
for prompt in prompts:
    print(prompt)
sentiments = score_sentiment(prompts, sentiment_tokenizer, sentiment_model, device)
print("")
print(f'individual sentiments: {sentiments[0:-1]}')
print(f'summed: {sentiments[0:-1].sum():0.2f}')
print(f'scored as a whole: {sentiments[-1]:0.2f}')

I felt good.
But I felt bad.
I felt good. But I felt bad.

individual sentiments: [ 0.95712024 -0.74740514]
summed: 0.21
scored as a whole: -0.10


In [9]:
prompts = ['I felt good.', 
               'Even though it was a terrible day.']

compound = ' '.join(prompts)
prompts.append(compound)
for prompt in prompts:
    print(prompt)
sentiments = score_sentiment(prompts, sentiment_tokenizer, sentiment_model, device)
print("")
print(f'individual sentiments: {sentiments[0:-1]}')
print(f'summed: {sentiments[0:-1].sum():0.2f}')
print(f'scored as a whole: {sentiments[-1]:0.2f}')

I felt good.
Even though it was a terrible day.
I felt good. Even though it was a terrible day.

individual sentiments: [ 0.95712042 -0.79817049]
summed: 0.16
scored as a whole: 0.50


In [10]:
prompts = ['I felt bad.', 
               'Even though it was a fun day.']

compound = ' '.join(prompts)
prompts.append(compound)
for prompt in prompts:
    print(prompt)
sentiments = score_sentiment(prompts, sentiment_tokenizer, sentiment_model, device)
print("")
print(f'individual sentiments: {sentiments[0:-1]}')
print(f'summed: {sentiments[0:-1].sum():0.2f}')
print(f'scored as a whole: {sentiments[-1]:0.2f}')

I felt bad.
Even though it was a fun day.
I felt bad. Even though it was a fun day.

individual sentiments: [-0.79355936  0.72371446]
summed: -0.07
scored as a whole: -0.39


In [11]:
prompts = ['It was great, we went out to lunch.',
           'I had a nice salad.',
           'Then she dumped me, and I felt sad.']

compound = ' '.join(prompts)
prompts.append(compound)
for prompt in prompts:
    print(prompt)
sentiments = score_sentiment(prompts, sentiment_tokenizer, sentiment_model, device)
print("")
print(f'individual sentiments: {sentiments[0:-1]}')
print(f'summed: {sentiments[0:-1].sum():0.2f}')
print(f'scored as a whole: {sentiments[-1]:0.2f}')

It was great, we went out to lunch.
I had a nice salad.
Then she dumped me, and I felt sad.
It was great, we went out to lunch. I had a nice salad. Then she dumped me, and I felt sad.

individual sentiments: [ 0.98594735  0.96157024 -0.86260593]
summed: 1.08
scored as a whole: 0.19
